Trong file này, em thực hiện cài đặt chương trình sử dụng DTW để nhận dạng khẩu lệnh đơn lẻ (mỗi từ/khẩu lệnh dùng khoảng 2-3 mẫu)

Để thực hiện công việc này, em sử dụng thư viện librosa.sequence.dtw và đọc thêm ở đây [Dynamic Time Warping Algorithm Review](https://www.researchgate.net/publication/228785661_Dynamic_Time_Warping_Algorithm_Review) để hiểu các thứ liên quan.

# Khai báo thư viện và các đường dẫn cần thiết

In [1]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import fnmatch
import pandas as pd
import math
from librosa.sequence import dtw
import copy
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount("/content/drive")
DATA_ROOT_DIR="/content/drive/MyDrive/Speech/Data_14/"
# !ls $DATA_ROOT_DIR

Mounted at /content/drive


# Chuẩn bị dữ liệu
Do mỗi label chỉ sử dụng 2-3 mẫu nên em thực hiện trích xuất MFCC từ một vài file âm thanh nhỏ luôn mà không thực hiện load lại list MFCC tổng hợp.

Công việc chuẩn bị dữ liệu bảo gồm:
- Định nghĩa hàm tính MFCC (như file tính MFCC)
- Chọn ra các mẫu dữ liệu và tính toán đặc trưng

In [3]:
def extract_mfcc_features(file_path,n_mfcc):
    #load file âm thanh
    sound, sr = librosa.load(file_path)

    #trích xuất đặc trưng
    mfcc = librosa.feature.mfcc(y=sound, sr=sr, n_mfcc=n_mfcc,hop_length=512)
    delta_mfcc = librosa.feature.delta(mfcc,width=9)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)

    #chuẩn hoá
    mfcc_features = np.concatenate((mfcc, delta_mfcc, delta2_mfcc)).T
    min_features = np.min(mfcc_features, axis=0)
    max_features = np.max(mfcc_features, axis=0)
    mfcc_features_nom = (mfcc_features - min_features) / (max_features - min_features)
    
    return mfcc_features_nom

In [4]:
#Chọn dữ liệu, ở đây mỗi label em sử dụng 4 samples
n_samples = 4
class_names = ["len", "xuong", "phai", "B", "A", "trai", "nhay", "ban"]
features = {}
labels = {}
sr=22050
n_mfcc=13
for cname in class_names:
    file_paths = [os.path.join("",DATA_ROOT_DIR, cname, ""+cname+"_"+str(i*15+1)+".wav") for i in range(n_samples)]
    data = [extract_mfcc_features(file_path,n_mfcc) for file_path in file_paths]
    features[cname] = data
    labels[cname] = [cname for i in range(len(file_paths))]

In [35]:
for cname in class_names:
    print(cname,len(features[cname]))

len 4
xuong 4
phai 4
B 4
A 4
trai 4
nhay 4
ban 4


In [5]:
data_arr = {}
for cname in class_names:
    data_arr[cname] = np.array(features[cname])
    for i in range(n_samples):
      data_arr[cname][i] = np.array(data_arr[cname][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


# Tạo ra template cho mỗi label
Trong phần này, em sử dụng DTW với nhiều samples và có thực hiện phân đoạn

In [6]:
model = {}

# Với mỗi label tạo ra 1 template
for cname in class_names:
  print(cname)
  # Khởi tạo một số tham số
  n_iter = 60 # số vòng lặp
  dif_update = 1 # sự cải tiến của template so với vòng lặp trước 
  epsilon = 1e-5 # điều kiện dừng của sự cải tiến
  n_seg = 5 # số đoạn phân
  template = np.array([data_arr[cname][0].copy(), data_arr[cname][1].copy(),
                       data_arr[cname][2].copy(), data_arr[cname][3].copy()]) 

  #Mảng lưu các vạch ngăn cách các đoạn, khởi tạo các đoạn chia đều
  bound = np.array([np.zeros(n_seg+1), np.zeros(n_seg+1), np.zeros(n_seg+1), np.zeros(n_seg+1)])
  for i in range(n_samples):
    len_temp = template[i].shape[0]
    bound[i][0] = math.ceil(0)
    bound[i][n_seg] = math.ceil(len_temp)
    for j in range(n_seg-1):
      bound[i][j+1] = math.ceil((j+1)*len_temp/n_seg)

  #Khởi tạo template (trung bình theo đoạn và giữa các samples)
  templateMean = np.array([np.zeros(39), np.zeros(39), np.zeros(39), np.zeros(39), np.zeros(39)])
  count = np.array([0, 0, 0, 0, 0])
  for j in range(n_seg):
    for i in range(n_samples):
      for k in range(int(bound[i][j]),int(bound[i][j+1])):
        templateMean[j] += template[i][k]
        count[j] += 1
    templateMean[j] /= count[j]

  #Khởi tạo về bước nhảy trạng thái
  step_sizes_sigma=np.array([[1, 1], [1, 0], [1, 2]])

  #Vòng lặp cập nhật template
  while (n_iter > 0 and dif_update > epsilon):

    #Với mỗi sample
    for i in range(n_samples):

      #dtw trả về mảng quy hoạch động chi phí và đường đi tối ưu
      D, wp = librosa.sequence.dtw(template[i].T, templateMean.T,step_sizes_sigma=step_sizes_sigma)
      
      #cập nhật các đoạn theo sự dóng hàng mới nhất
      for j in range(n_seg-1):
        for m in range(template[i].shape[0]-1):
          if (wp[m][1] == n_samples-j and wp[m+1][1] == n_samples-j-1):
            bound[i][n_samples-j]=wp[m][0]
    
    #cập nhật template trung bình
    old_temp = copy.deepcopy(templateMean)
    templateMean = np.array([np.zeros(39), np.zeros(39), np.zeros(39), np.zeros(39), np.zeros(39)])
    count = np.array([0, 0, 0, 0, 0])
    for j in range(n_seg):
      for i in range(n_samples):
        for k in range(int(bound[i][j]),int(bound[i][j+1])):
          templateMean[j] += template[i][k]
          count[j] += 1
      templateMean[j] /= count[j]
    n_iter-=1  

    #Tính toán sự cải thiện so với vòng lặp trước
    dif_update = np.sqrt(np.linalg.norm(templateMean - old_temp, ord="fro"))

  #Lưu lại template thu được cuối cùng
  model[cname] = templateMean

len


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if sys.path[0] == '':


xuong
phai
B
A
trai
nhay
ban


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if sys.path[0] == '':


# Đánh giá mô hình bằng tập test
Trong phần này em thực hiện đánh giá mô hình. Em chọn ngẫu nhiên một số lượng nhỏ dữ liệu để quan sát kết quả. Nhãn dự đoán là nhãn ứng với template có chi phí dóng hàng nhỏ nhất

In [7]:
test_mfcc = []
test_labels = []

for cname in class_names:
    file_paths = [os.path.join("",DATA_ROOT_DIR, cname, ""+cname+"_"+str(i*100+5)+".wav") for i in range(n_samples)]
    data = [extract_mfcc_features(file_path,n_mfcc) for file_path in file_paths]
    for i in range(len(data)):
      test_mfcc.append(data[i])
      test_labels.append(cname)

In [8]:
y_pred = []

for i in range(0,len(test_mfcc)):
    cost = {}

    #Dóng hàng input với các template và chọn ra template có chi phí nhỏ nhất
    for cname in class_names:
      D,wp = librosa.sequence.dtw(test_mfcc[i].T, model[cname].T,step_sizes_sigma=step_sizes_sigma)
      cost[cname] = D[test_mfcc[i].T.shape[1]-1][model[cname].T.shape[1]-1]
    pred = min(cost, key=lambda k: cost[k])
    y_pred.append(pred)

In [9]:
#In kết quả quan sát
report = classification_report(test_labels, y_pred, target_names=class_names)
print(report)

              precision    recall  f1-score   support

         len       1.00      1.00      1.00         4
       xuong       0.75      0.75      0.75         4
        phai       1.00      0.50      0.67         4
           B       0.50      0.75      0.60         4
           A       1.00      0.50      0.67         4
        trai       0.57      1.00      0.73         4
        nhay       1.00      0.75      0.86         4
         ban       1.00      1.00      1.00         4

    accuracy                           0.78        32
   macro avg       0.85      0.78      0.78        32
weighted avg       0.85      0.78      0.78        32



In [10]:
#quan sát tập nhãn dự đoán so với nhãn đúng
print(test_labels)
print(y_pred)

['len', 'len', 'len', 'len', 'xuong', 'xuong', 'xuong', 'xuong', 'phai', 'phai', 'phai', 'phai', 'B', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'trai', 'trai', 'trai', 'trai', 'nhay', 'nhay', 'nhay', 'nhay', 'ban', 'ban', 'ban', 'ban']
['len', 'len', 'B', 'len', 'xuong', 'xuong', 'xuong', 'xuong', 'phai', 'phai', 'phai', 'phai', 'B', 'B', 'B', 'phai', 'A', 'A', 'A', 'A', 'trai', 'trai', 'phai', 'trai', 'nhay', 'nhay', 'phai', 'len', 'ban', 'ban', 'len', 'len']


# Demo nhận dạng với đoạn dữ liệu mới thu

In [17]:
audio_file = "/content/drive/MyDrive/Speech/Data/len.wav"
ipd.Audio(audio_file)
# load audio files with librosa
signal, sr = librosa.load(audio_file)
mfccs_features = extract_mfcc_features(audio_file,n_mfcc)
mfccs_features.shape
# mfccs_features

(30, 39)

In [18]:
cost = {}
for cname in class_names:
  D,wp = librosa.sequence.dtw(mfccs_features.T, model[cname].T,step_sizes_sigma=step_sizes_sigma)
  # print(D.shape)
  cost[cname] = D[mfccs_features.T.shape[1]-1][model[cname].T.shape[1]-1]
  # print(mfccs_features.T.shape)
  # print(model[cname].T.shape)
print(cost)

{'len': 41.09538200078585, 'xuong': 57.99488071578956, 'phai': 60.162707138637025, 'B': 54.30067037276879, 'A': 59.714654877874096, 'trai': 57.10609964872166, 'nhay': 52.51076121931237, 'ban': 58.827587189663156}


In [19]:
pred = min(cost, key=lambda k: cost[k])
print(pred)

len
